In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

# 1. 평가 데이터셋 준비 (예시 데이터 2개)
eval_data = [
    {
        "id": 2,
        "question": "공동개발 가이드 문서 검수는 누가 해?",
        "gt_chapter": "Ch.1",
        "gt_scene": "Scene 018",
        "gt_answer": "맥스",
        "raw_log": """
🔍 [분석 시작]: '공동개발 가이드 문서 검수는 누가 해?'
📡 [Step 1] 관련 챕터 검색 (Top-3)...
   - Ch.1 (유사도: 0.4413)
   - Ch.17 (유사도: 0.2660)
📡 [Step 2] 앵커 씬 검색 (Top-5)...
   - Scene 018, Scene 015...
🎯 AI 답변: 공동 개발 가이드 문서의 검수는 엔디 님이 맡고 있습니다.
        """
    },
    # 여기에 50개 데이터를 추가하시면 됩니다.
]

df = pd.DataFrame(eval_data)

In [ ]:
def evaluate_rag(row):
    log = row['raw_log']

    # [Step 1] Retrieval 성공 여부 체크 (Chapter)
    retrieved_chapters = re.findall(r'Ch\.\d+', log)
    retrieval_success = 1 if row['gt_chapter'] in retrieved_chapters else 0

    # [Step 2] Generation 성공 여부 (단답형 EM 기준)
    # 실제로는 텍스트 유사도나 LLM 채점을 권장하지만, 여기서는 단순 포함 여부로 체크
    ai_answer = log.split("🎯 AI 답변:")[1].strip() if "🎯 AI 답변:" in log else ""
    generation_success = 1 if row['gt_answer'] in ai_answer else 0

    # 최종 성공 (둘 다 성공해야 1)
    total_success = 1 if retrieval_success == 1 and generation_success == 1 else 0

    return pd.Series([retrieval_success, generation_success, total_success, ai_answer])

# 평가 실행
df[['ret_ok', 'gen_ok', 'total_ok', 'ai_response']] = df.apply(evaluate_rag, axis=1)

In [ ]:
def classify_error(row):
    if row['ret_ok'] == 0: return 'Retrieval Error'
    elif row['gen_ok'] == 0: return 'Generation Error'
    else: return 'Success'

df['error_type'] = df.apply(classify_error, axis=1)
error_counts = df['error_type'].value_counts()

plt.figure(figsize=(8, 8))
plt.pie(error_counts, labels=error_counts.index, autopct='%1.1f%%', startangle=140, colors=['#ff9999','#66b3ff','#99ff99'])
plt.title('Error Analysis Distribution')
plt.show()

In [ ]:
# 지표 계산
total_count = len(df)
ret_accuracy = df['ret_ok'].mean() * 100
gen_accuracy = df['gen_ok'].mean() * 100
final_accuracy = df['total_ok'].mean() * 100

# 시각화 코드
plt.figure(figsize=(10, 6))
sns.set_style("whitegrid")

metrics = ['Retrieval Success', 'Generation Success', 'Total Accuracy']
values = [ret_accuracy, gen_accuracy, final_accuracy]

colors = ['#4A90E2', '#50E3C2', '#F5A623']
bars = plt.bar(metrics, values, color=colors)

# 수치 표시
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, yval + 1, f'{yval:.1f}%', ha='center', va='bottom', fontsize=12, fontweight='bold')

plt.ylim(0, 110)
plt.title('Multimodal Webtoon RAG Performance Evaluation', fontsize=15)
plt.ylabel('Success Rate (%)')
plt.show()